In [2]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
from sklearn.metrics import f1_score,confusion_matrix
from torch.nn import functional

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [1]:
import pickle

with open("./pickle_data/balanced_data.pickle", "rb") as f:
    balenced_data = pickle.load(f)
    
with open("./pickle_data/token_data.pickle", "rb") as f:
    token_data = pickle.load(f)

In [8]:
comment = balenced_data["comment"]
target = balenced_data["target"]
reviews_idx = token_data["comment_ix"]
word2idx = token_data["word2ix"]
idx2word = token_data["ix2word"]
max_seq_length = token_data["max_seq_length"]
idx2target={1:'highlight',0:'no'}

In [9]:
a=[]
for i,k in zip(comment,target):
    a.append([i,k])

In [22]:
len(comment)

20343

In [11]:
b=[]
for i in range(0,len(comment)):
    if a[i][1]==1:
        b.append(a[i])

In [12]:
b[0][0]

['킹/Noun', '킹민철/Noun']

In [13]:
li=[]
for k in range(0,len(b)):
    li.append(b[k][0])

In [14]:
len(b)a

9932

In [15]:
target_non=[]
for k in range(0,10622):
    target_non.append(0)

In [20]:
len(target_non)

10622

In [16]:
X_train,comment_non,y_train,y_test=train_test_split(li,target_non,test_size=0.3)

ValueError: Found input variables with inconsistent numbers of samples: [9932, 10622]

In [17]:
len(comment_non)

NameError: name 'comment_non' is not defined

In [90]:
target_high=[]
for k in range(0,10121):
    target_high.append(0)

In [81]:
len(target_high)

10121

In [83]:
len(b)

10121

In [91]:
for k in target_high:
    target_non.append(k)


In [68]:
for k in target_high:
    target_non.append(k)
for k in b:
    comment_non.append(k)

In [70]:
comment=comment_non.copy()

In [84]:
len(comment)

20743

In [92]:
target=target_non.copy()

In [93]:
len(target)

20743

In [94]:
def padding(token):
    token_post=token.copy()
    len_sentence=[ len(i) for i in token_post]
    max_sentence=max(len_sentence) #최대 문장길이 9591
    n=-1
    for i in token_post:
        n+=1
        k=len(token_post[n])
        while max_sentence-k>=0:
            token_post[n].append('<PAD>')
            k+=1
    return token_post

In [95]:
comment=list(map(lambda x: padding(x),  [comment]))
comment=comment[0]

In [96]:
len(comment)

20743

In [103]:
word2idx['']

KeyError: '옹'

In [107]:
for i in comment:
    for k in i:
        li1.append(word2idx[k])

TypeError: unhashable type: 'list'

In [100]:
li1=[
comment_list=[]
for i in comment:
    for k in i:
        li1.append(word2idx[k])
    
    comment_list.append(li1)
    li1=[]

TypeError: unhashable type: 'list'

In [97]:
X_train,X_test,y_train,y_test=train_test_split(comment,target,test_size=0.3,random_state=7,stratify=target)

ValueError: Found input variables with inconsistent numbers of samples: [10622, 20743]

In [20]:
from gensim.models import Word2Vec
model = Word2Vec.load('word2vec_50_total.model')

In [21]:
import numpy as np
import torch

embedding_matrix = []

for word in word2idx.keys():
    try:
        embedding_matrix.append(model[word])
    except:
        embedding_matrix.append(np.zeros(50))

print(len(word2idx))
print(len(embedding_matrix))

embedding_matrix = torch.Tensor(embedding_matrix)

C:\Users\jimmi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


17786
17786


In [22]:
embedding_matrix

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6848,  1.4290, -1.4527,  ..., -0.7468, -1.0116,  0.8647],
        ...,
        [-0.0051, -0.0071,  0.0023,  ...,  0.0030, -0.0024,  0.0044],
        [ 0.5481,  0.0921, -1.0365,  ...,  0.1465,  0.0914,  0.6699],
        [-2.2798,  0.3819,  0.4305,  ..., -0.5126, -0.5356, -0.6259]])

In [23]:
batch_size=len(X_train)

In [24]:
class GRU_Clf(nn.Module):
    def __init__(self, input_size, embedding_size,num_class,num_layers,hidden_size,bidirectional,num_directions):
        
        super(GRU_Clf,self).__init__()
        
        # 안에 하이퍼 파라미터의 종류들 선언
        
        self.num_layers=num_layers
        self.hidden_size=hidden_size
        self.input_size=input_size
        self.embedding_size=embedding_size
        self.num_class=num_class
        self.num_directions=num_directions
        
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix,freeze=False)
        
        self.gru=nn.GRU(input_size=embedding_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        bidirectional=bidirectional) # 해당 sequence의 앞 뒤를 모두 고려해서 반영
        
        self.linear=nn.Linear(in_features=hidden_size*num_directions,out_features=num_class)
   

    
    def forward(self,inputs):
        
        hidden=self.init_hidden(batch_size)
        
        embed=self.embedding(inputs)
        
        out, _ =self.gru(embed,hidden)
        
        return self.linear(out[:, -1:, :].squeeze(1))
    
    
    def predict(self,inputs):
        
        hidden = self.init_hidden(1)
        embed = self.embedding(inputs)

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        
        out, _ = self.gru(embed, hidden)

        return self.linear(out[:, -1:, :].squeeze(1))
    
    def init_hidden(self, batch_size):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        return Variable(torch.zeros(self.num_layers*self.num_directions,batch_size, self.hidden_size))
    
    

In [25]:
len(X_train[0])

79

In [26]:
batch_size=len(X_train)
#bidirectional=False


model = GRU_Clf(input_size=50,num_directions=1, embedding_size = 50,hidden_size=len(X_train[0]),
                num_layers=1,num_class=2,bidirectional=False).to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.1,momentum=0.9)

for epoch in range(5):

    model.zero_grad()
    inputs = Variable(torch.LongTensor(X_train)).to(device)
    targets = Variable(torch.LongTensor(y_train)).to(device)
    
    preds = model(inputs)
    
    loss = loss_function(preds, targets)

    if epoch % 1 == 0:
        print(loss.item())
    
    loss.backward()
    optimizer.step()

0.6990488767623901
0.6673503518104553
0.6187829971313477
0.5716196894645691
0.5402790307998657


In [27]:
correct = 0
list_predicted=[]
model.eval()
for i, seq in enumerate(X_test):
    input = Variable(torch.LongTensor(seq).view(1,-1)).to(device)
    pred = model.predict(input)
    _, pred = torch.max(pred, 1)
    list_predicted.append(pred)
    true = y_test[i]
    if true == pred.item():
        correct +=1
    
    if i%10000 == 0:
        input_seq = [idx2word.get(ix) for ix in seq if ix != 0]
        print("Input :", input_seq)
        print("Prediction :", pred.item())
        print("Truth :", y_test[i])
        print("\n")

print("Accuracy :%s" %(correct/len(X_test)*100))
print('F1 score:%s' %(f1_score(list_predicted,y_test)))

Input : ['무조건/Noun', '사이드/Noun', '쌘거/Noun', '카밀/Noun']
Prediction : 0
Truth : 0


Input : ['이블린/Noun', '피/Noun', 'ㅅㄱ/KoreanParticle']
Prediction : 0
Truth : 0


Accuracy :77.77126958559086


C:\Users\jimmi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


F1 score:0.0


In [28]:
print(confusion_matrix(list_predicted,y_test))

[[10622  3036]
 [    0     0]]
